In [120]:
import numpy as np
import scipy.stats as scps
from scipy.special.orthogonal import ps_roots
import scipy.interpolate as scin
from copy import *
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
from ret import (
    util,
    egm_step
)
from aux import create_container

In [122]:
import pickle
with open('m0_value_new.pkl', 'rb') as file : 
    m0_value = pickle.load(file)

with open('m0_policy_new.pkl', 'rb') as file : 
    m0_policy = pickle.load(file)


# Model parameters (default)

In [100]:
# Number of periods (fist period is t=1) 
Tbar = 25

# Number of grid points over assets
ngridm = 500

# Maximum level of assets
mmax = 50

# Number of quadrature points used in calculation of expectations
n_quad_points = 5

# Number of simulations
nsims = 50

# Interval of the initial wealth
init = [10, 30]

# Interest rate
r = 0.05

# Discount factor
beta = 0.95

# Standard deviation of log-normally distributed income shocks
sigma = 0.00

# Disutility of work
duw = 0.35

# CRRA coefficient (log utility if ==1)
theta = 1.95

# Careful with the coefficients here -- original code had the polynomial
# Coded as a + b * x - c * x**2 ... (note the crazy minus)
coeffs_age_poly = np.array([0.75, 0.04, -0.0002])

# Consumption floor (safety net in retirement)
cfloor = 0.001

# Scale of the EV taste shocks 
lambda_ = 2.2204e-16

In [101]:
# Initialize grids
quadstnorm = scps.norm.ppf(ps_roots(n_quad_points)[0])
quadw = ps_roots(n_quad_points)[1]

In [102]:
savingsgrid = np.linspace(0, mmax, ngridm)

In [103]:
policy, value = create_container(ngridm, Tbar, mmax, theta, duw)

In [104]:
for period in [23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]:
    value, policy = egm_step(value, policy, savingsgrid, quadstnorm, period, Tbar, ngridm, cfloor, n_quad_points, r, coeffs_age_poly, theta, duw, beta, lambda_, sigma, quadw)

In [105]:
for point in [23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9,  8, 7, 6, 5, 4, 3, 2, 1, 0]:
    np.testing.assert_almost_equal(m0_value[:, 0, 1, point].T[~np.isnan(m0_value[:, 0, 1, point])], value[point][1].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_value[:, 1, 1, point].T[~np.isnan(m0_value[:, 1, 1, point])], value[point][1].T[1], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 0, 1, point].T[~np.isnan(m0_policy[:, 0, 1, point])], policy[point][1].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 1, 1, point].T[~np.isnan(m0_policy[:, 1, 1, point])], policy[point][1].T[1], decimal=8)
    np.testing.assert_almost_equal(m0_value[:, 0, 0, point].T[~np.isnan(m0_value[:, 0, 0, point])], value[point][0].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_value[:, 1, 0, point].T[~np.isnan(m0_value[:, 1, 0, point])], value[point][0].T[1], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 0, 0, point].T[~np.isnan(m0_policy[:, 0, 0, point])], policy[point][0].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 1, 0, point].T[~np.isnan(m0_policy[:, 1, 0, point])], policy[point][0].T[1], decimal=8)


In [106]:
from dcegm.solve.solve import solve_retirement_model

In [107]:
value, policy_ = solve_retirement_model(ngridm, n_quad_points, r, coeffs_age_poly, theta, duw, beta, lambda_, sigma, mmax, Tbar)

In [126]:
for period in range(24):
    for choice in [1,0]:
        np.testing.assert_array_almost_equal(value_[period][choice], m0_value[period][choice])

In [128]:
m0_value[24]

[array([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]]), array([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]])]

In [115]:
m0_policy

[[array([[0.00000000e+00, 1.00128448e-03, 1.07874813e-01, ...,
          5.34561165e+01, 5.35636741e+01, 5.36712316e+01],
         [0.00000000e+00, 1.00128448e-03, 7.67441241e-03, ...,
          3.65651728e+00, 3.66387446e+00, 3.67123163e+00]]),
  array([[ 0.        ,  4.49553039,  4.70077341,  4.80833099,  4.91588857,
           5.02344615,  5.13100373,  5.2385613 ,  5.34611888,  5.45367646,
           5.56123404,  5.66879162,  5.7763492 ,  5.88390678,  5.99146435,
           6.09902193,  6.20657951,  6.31413709,  6.42169467,  6.52925225,
           6.63680982,  6.7443674 ,  6.85192498,  6.95948256,  6.99909208,
           6.99909208,  7.05559147,  7.16314905,  7.27070663,  7.37826421,
           7.48582179,  7.59337936,  7.70093694,  7.80849452,  7.9160521 ,
           8.02360968,  8.13116726,  8.23872483,  8.34628241,  8.45383999,
           8.56139757,  8.66895515,  8.77651273,  8.8840703 ,  8.99162788,
           9.09918546,  9.15665105,  9.15665105,  9.18993255,  9.29749013,
    